In [91]:
import numpy as np

def CTS():
    s = np.matrix([[1, 1],[1, 1]])
    f = np.matrix([[1, 1],[1, 1]])
    
    T = 1000
    N = 1000
    N_obs = 200

    theta = np.matrix([[0.1, 0.5, 0.4, 0.2],
            [0.5, 0.1, 0.2, 0.4]])
    pObs  = np.matrix([[(1-theta[0, 0] + 1-theta[0, 3]) * N_obs / 4, (theta[0, 0] + theta[0, 3]) * N_obs / 4], [(1-theta[1, 1] + 1-theta[1, 2]) * N_obs / 4, (theta[1, 1] + theta[1, 2]) * N_obs / 4]])

    p_X = [(pObs[0,0]+pObs[0,1]) / (sum(pObs)[0,0]+sum(pObs)[0,1]), (pObs[1,0]+pObs[1,1]) / (sum(pObs)[0,0]+sum(pObs)[0,1])]
    p_Y_X = [pObs[0, 1] / (pObs[0,0]+pObs[0,1]), pObs[1, 1] / (pObs[1,0]+pObs[1,1])]
    zCount = [0, 0]

    s[0, 0] = pObs[0, 1]
    s[1, 1] = pObs[1, 1]
    f[0, 0] = pObs[0, 0]
    f[1, 1] = pObs[1, 0]

    Action = np.zeros((1, T))
    Reward = np.zeros((1, T))
    Prob   = np.zeros((1, T))
    Conds  = [0, 0, 0, 0]

    currentFactors = np.zeros((3, T))
    for i in range(T):
        B = np.random.binomial(1, 0.5)
        D = np.random.binomial(1, 0.5)
        Z = (B^D) + 1
        currentFactors[:,i] = [B, D, Z]

    for t in range(T):
        roundFactors = currentFactors[:,t]
        B = int(roundFactors[0])
        D = int(roundFactors[1])
        Z = int(roundFactors[2])
        zPrime = 3 - Z
        covariateIndex = int(B + D * 2)
        Conds[covariateIndex] = Conds[covariateIndex] + 1

        zCount[Z-1] = zCount[Z-1] + 1
        p_Z = [zCount[0] / sum(zCount), zCount[1] / sum(zCount)]
        p_Y_doX_Z = np.matrix([[s[0,0] / (s[0,0] + f[0,0]), s[0,1] / (s[0,1] + f[0,1])],
                    [s[1,0] / (s[1,0] + f[1,0]), s[1,1] / (s[1,1] + f[1,1])]])

        
        Q1 = p_Y_doX_Z[Z-1, zPrime-1]
    
        Q2 = p_Y_X[Z-1]

        bias = abs(Q1 - Q2)
        w = [1, 1]
        if math.isnan(bias):
            weighting = 1
        else:
            weighting = 1 - bias
        if Q1 > Q2:
            w[Z-1] = weighting
        else:
            w[zPrime] = weighting
    
        theta_hat = [np.random.beta(s[Z-1, 0], f[Z-1, 0], 1)[0] * w[0], np.random.beta(s[Z-1, 1], f[Z-1, 1]) * w[1]]
        maxVal = max(theta_hat)
        action = theta_hat.index(max(theta_hat))
        currentTheta = theta[action, covariateIndex]

        reward = (np.random.rand() <= currentTheta)

        s[Z-1, action] = s[Z-1, action] + reward
        f[Z-1, action] = f[Z-1, action] + 1 - reward

        Action[0,t] = (action == 0)
        Reward[0,t] = reward
        bestVal = max([theta[0, covariateIndex], theta[1, covariateIndex]])
        bestAction = [theta[0, covariateIndex], theta[1, covariateIndex]].index(max([theta[0, covariateIndex], theta[1, covariateIndex]]))
        Prob[0,t] = (action == bestAction)

    return Action, Reward, Prob



    
    

In [92]:
a, r, p = CTS()